## Dependencies

In [1]:
from ast import Add
from ast import Assign
from ast import BinOp
from ast import Call
from ast import Constant
from ast import Div
from ast import dump
from ast import Expr
from ast import fix_missing_locations
from ast import Load
from ast import Module
from ast import Mult
from ast import Name
from ast import Not
from ast import operator
from ast import parse
from ast import stmt
from ast import Store
from ast import Sub
from ast import UnaryOp
from dis import dis
from dis import show_code
from typing import Callable


## Basic implementation


In [2]:
source = """
a = 0
"""

node = parse(source, mode="exec")
dump(node)

compiled = compile(node, "<string>", mode="exec")
# exec(compiled)

dis(compiled)
print("---")
show_code(compiled)


  0           0 RESUME                   0

  2           2 LOAD_CONST               0 (0)
              4 STORE_NAME               0 (a)
              6 LOAD_CONST               1 (None)
              8 RETURN_VALUE
---
Name:              <module>
Filename:          <string>
Argument count:    0
Positional-only arguments: 0
Kw-only arguments: 0
Number of locals:  0
Stack size:        1
Flags:             0x0
Constants:
   0: 0
   1: None
Names:
   0: a


## Laboratory


### Helpers

In [3]:
OPERATOR_MAP = {
    "=": Assign,  # Has a different behavior
    "*": Mult,
    "/": Div,
    "+": Add,
    "-": Sub,
    "not": Not,
}


In [4]:
def get_operator(symbol: str) -> operator:
    """Retrieves an operator function from a map based on the provided symbol."""

    try:
        return OPERATOR_MAP[symbol]
    except KeyError as exception:
        raise ValueError(f"Unsupported operator: {operator}") from exception


def get_value(value: str | int) -> str | int:
    """Attempts to convert the input to an integer. If the conversion fails, the original input is returned."""

    try:
        return int(value)
    except ValueError:
        return value
    

def is_valid_line(line: str) -> bool:
    """Determines if a line of code is valid (i.e., not empty and not a comment)."""

    return line.strip() and not line.strip().startswith("#")

### Functions

In [5]:
def build_print_operation(operand: str | int) -> Expr:
    operand = get_value(operand)
    if isinstance(operand, str):
        args = [Name(id=operand, ctx=Load())]
    elif isinstance(operand, int):
        args = [Constant(value=operand)]
    else:
        raise TypeError("Only str and int types are supported")
    return Expr(value=Call(func=Name(id='print', ctx=Load()), args=args, keywords=[]))


def build_unary_operation(operator: str, operand: str) -> Expr:
    operator = get_operator(operator)
    operand = Name(id=operand, ctx=Load())
    binop = UnaryOp(op=operator(), operand=operand)
    return Expr(value=binop)


def build_binary_operation(operand_1: str, operator: str, operand_2: str) -> Expr:
    operand_1 = Name(id=operand_1, ctx=Load())
    operand_2 = Name(id=operand_2, ctx=Load())
    operator = get_operator(operator)
    binop = BinOp(left=operand_1, op=operator(), right=operand_2)
    return Expr(value=binop)


def build_assignation_operation(targets: list[str], value: int | str | stmt) -> Assign:
    targets = [Name(id=target.strip(), ctx=Store()) for target in targets]
    if isinstance(value, (int, str)):
        value = Constant(value=value)
        return Assign(targets=targets, value=value)
    return Assign(targets=targets, value=value)


In [6]:

def get_operation_generator(terms: int) -> Callable:
    function_map = {
        "1": build_print_operation,
        "2": build_unary_operation,
        "3": build_binary_operation,
    }
    try:
        return function_map[str(terms)]
    except KeyError as exception:
        raise ValueError(f"Unsupported number of terms") from exception


In [7]:
def get_line_expression(raw_expression: str, is_assignment: bool = False) -> stmt:
    # if "=" in raw_expression:  # check if the expression is an assignment
    #     targets_str, value_str = raw_expression.split("=")
    #     targets = targets_str.split(",")
    #     value = get_line_expression(value_str, is_assignment=True)
    #     return build_assignation_operation(targets, value)

    # terms = raw_expression.split()
    # num_terms = len(terms)
    # if is_assignment and num_terms == 1:
    #     return get_value(terms[0])

    # return get_operation_generator(num_terms)(*terms)

    for operator in OPERATOR_MAP:
        if operator in raw_expression:
            break
        return build_print_operation(raw_expression.strip())
    return parse(raw_expression).body[0]


def build_ast(raw_expression: str) -> None:
    lines = raw_expression.split("\n")
    expressions = [get_line_expression(line) for line in lines if is_valid_line(line)]
    module = Module(body=expressions, type_ignores=[])
    print(dump(module))

    fix_missing_locations(module)
    code = compile(module, '<string>', mode='exec')
    exec(code)
        

### Demonstration

In [8]:
formula = """
a=23
b=1
c=b
d=a+b+c
d
"""

In [9]:
build_ast(formula)

Module(body=[Assign(targets=[Name(id='a', ctx=Store())], value=Constant(value=23)), Assign(targets=[Name(id='b', ctx=Store())], value=Constant(value=1)), Assign(targets=[Name(id='c', ctx=Store())], value=Name(id='b', ctx=Load())), Assign(targets=[Name(id='d', ctx=Store())], value=BinOp(left=BinOp(left=Name(id='a', ctx=Load()), op=Add(), right=Name(id='b', ctx=Load())), op=Add(), right=Name(id='c', ctx=Load()))), Expr(value=Call(func=Name(id='print', ctx=Load()), args=[Name(id='d', ctx=Load())], keywords=[]))], type_ignores=[])
25
